In [ ]:
# |default_exp gbx

In [ ]:
# |export
import json
import logging
import re
import unicodedata
import uuid
from functools import cache
from itertools import chain
from pathlib import Path, PureWindowsPath
from subprocess import check_output

from fastcore.basics import store_attr
from fastcore.foundation import L, Self
from fastcore.net import do_request, urlread
from tenacity import retry, wait_exponential

from tmenv import __file__ as tmenv_root

tmenv_root = Path(tmenv_root).parent

# Gbx
> Retrieve and parse map and ghost gbx files

In [ ]:
# |export
API_HEADERS = {"User-Agent": "pedroaitm"}


def gbx_bytes_to_json(gbx_bytes):
    path = Path(f"/tmp/trackmania/{uuid.uuid1()}.gbx")
    path.parent.mkdir(exist_ok=True, parents=True)
    with open(path, "wb") as f:
        f.write(gbx_bytes)
    cmd = f'{tmenv_root}/gbx-to-json/GbxToJson "{path}"'
    return json.loads(check_output(cmd, encoding="utf-8", shell=True))


@cache
@retry(wait=wait_exponential(multiplier=1, min=1, max=900))
def gbx_data_from_url(url):
    gbx_bytes = urlread(url, headers=API_HEADERS, decode=False)
    return gbx_bytes_to_json(gbx_bytes)


def gbx_data_from_file(path):
    with open(path, "rb") as f:
        return gbx_bytes_to_json(f.read())


def gbx_time_to_ms(time):
    return time["TotalMilliseconds"]


def parse_sample(sample_data):
    return dict(
        racetime=gbx_time_to_ms(sample_data["Time"]),
        position=[sample_data["Position"][a] for a in "XYZ"],
        rotation=[sample_data["Rotation"][a] for a in "XYZW"],
        pitch=sample_data["PitchYawRoll"]["X"],
        yaw=sample_data["PitchYawRoll"]["Y"],
        roll=sample_data["PitchYawRoll"]["Z"],
        speed=sample_data["Speed"],
        velocity=[sample_data["Velocity"][a] for a in "XYZ"],
        ground_contact=sample_data["IsGroundContact"],
    )


class CorruptedGhostException(Exception):
    pass


def parse_ghost(ghost_data):
    if not ghost_data:
        return dict()
    if "Node" in ghost_data:
        ghost_data = ghost_data["Node"]
    ent_list = L(ghost_data["RecordData"]["EntList"]).filter(
        lambda s: s["Type"] in [2, 4, 6]
    )
    if ent_list.map(Self["Samples"]).map(bool).sum() != 1:
        raise CorruptedGhostException(
            "Samples were found in 0 or more than 1 buffer type"
        )
    for ent in ent_list:
        if ent["Samples"]:
            samples = L(ent["Samples"]).map(parse_sample)
            break
    if samples.sorted(lambda s: s["racetime"]) != samples:
        raise CorruptedGhostException("Samples or not ordered")
    return dict(
        racetime=gbx_time_to_ms(ghost_data["RaceTime"]),
        checkpoints=list(
            L(ghost_data["Checkpoints"]).map(Self["Time"]).map(gbx_time_to_ms)
        ),
        samples=list(samples),
    )


def sanitize_map_name(map_name):
    return re.sub(r"\$...", "", map_name)


def parse_map(map_data):
    if "Node" in map_data:
        map_data = map_data["Node"]
    ghosts = dict()
    if map_data["ChallengeParameters"]["RaceValidateGhost"]:
        ghosts["author"] = parse_ghost(
            map_data["ChallengeParameters"]["RaceValidateGhost"]
        )
    return dict(
        name=map_data["MapName"],
        sanitized_name=sanitize_map_name(map_data["MapName"]),
        uid=map_data["MapUid"],
        nb_checkpoints=map_data["NbCheckpoints"],
        medals=dict(
            bronze=gbx_time_to_ms(map_data["TMObjective_BronzeTime"]),
            silver=gbx_time_to_ms(map_data["TMObjective_SilverTime"]),
            gold=gbx_time_to_ms(map_data["TMObjective_GoldTime"]),
            author=gbx_time_to_ms(map_data["TMObjective_AuthorTime"]),
        ),
        ghosts=ghosts,
    )

In [ ]:
# |export
@cache
def map_url_from_uid(uid):
    req_url = f"https://trackmania.io/api/map/{uid}"
    return do_request(req_url, headers=API_HEADERS)["fileUrl"]


def map_from_file(path):
    return parse_map(gbx_data_from_file(path))


def ghost_wr_url_from_map_uid(uid):
    req_url = f"https://trackmania.io/api/leaderboard/map/{uid}"
    ghost_url = do_request(req_url, headers=API_HEADERS)["tops"][0]["url"]
    return f"https://trackmania.io{ghost_url}"


def ghost_wr_from_map_uid(uid):
    req_url = f"https://trackmania.io/api/leaderboard/map/{uid}"
    tops = do_request(req_url, headers=API_HEADERS)["tops"]
    for i, top in enumerate(tops):
        data = gbx_data_from_url(f"https://trackmania.io{top['url']}")
        try:
            return parse_ghost(data)
        except CorruptedGhostException:
            logging.warning(f"Map {uid} top {i+1} replay is corrupted")
    logging.warn(f"Map {uid}, no valid ghost found in the top {len(tops)}")


def ghost_from_url(url):
    return parse_ghost(gbx_data_from_url(url))


def ghost_from_file(path):
    return parse_ghost(gbx_data_from_file(path))

In [ ]:
# |export
@cache
def map_from_url(url, fetch_wr=True):
    result = parse_map(gbx_data_from_url(url))
    result["url"] = url
    if fetch_wr:
        wr = ghost_wr_from_map_uid(result["uid"])
        if wr is not None:
            result["ghosts"]["wr"] = wr
    return result


@cache
def map_from_uid(uid, fetch_wr=True):
    result = map_from_url(map_url_from_uid(uid), fetch_wr)
    assert uid == result["uid"]
    return result

In [ ]:
# |export
@cache
def official_campaign(id):
    return do_request(
        f"https://trackmania.io/api/officialcampaign/{id}", headers=API_HEADERS
    )


@cache
def official_campaigns():
    return (
        L(
            do_request(f"https://trackmania.io/api/campaigns/0", headers=API_HEADERS)[
                "campaigns"
            ]
        )
        .filter(Self["clubid"], negate=True)
        .map(Self["id"])
        .map(official_campaign)
        .sorted(Self["publishtime"])
    )


def campaign_maps_uid(campaign):
    return L(campaign["playlist"]).map(Self["mapUid"])


def campaigns_maps_uid(campaigns, slice):
    uids = chain.from_iterable(
        campaign_maps_uid(campaign)[slice] for campaign in campaigns
    )
    return L(uids)


@cache
def campaign(club_id, campaign_id):
    return do_request(
        f"https://trackmania.io/api/campaign/{club_id}/{campaign_id}",
        headers=API_HEADERS,
    )

In [ ]:
# |hide
# |eval: false
from nbdev.doclinks import nbdev_export

nbdev_export()